# Multiclass classification

In [1]:
import pennylane as qml
from pennylane import numpy as np
import torch
from torch.autograd import Variable
import torch.optim as optim

In [52]:
np.random.seed(0)
torch.manual_seed(0)

num_qubits = 4
num_classes = 3
dev = qml.device("default.qubit", wires = num_qubits)

### Attempt 1 - Angle Encoding and multiplied square loss

In [53]:
from sklearn import datasets

def load_and_process_data_two_labels():
    iris = datasets.load_iris()
    X = iris.data
    Y = iris.target
    Y1 = np.zeros((len(Y), 2))
    
    for i in range(len(Y)):
        if Y[i] == 0:
            Y1[i] = [0,0]
        elif Y[i] == 1:
            Y1[i] = [0,1]
        elif Y[i] == 2:
            Y1[i] = [1,0]
        elif Y[i] == 3:
            Y1[i] = [1,1]
    
    return X, Y1

### 2. Quantum Circuit

In [58]:
from pennylane.templates import AmplitudeEmbedding
from pennylane.templates.layers import StronglyEntanglingLayers

@qml.qnode(dev)
def quantum_circuit(weights, x):
    AmplitudeEmbedding(x, wires = [0,1], normalize = True)  
    StronglyEntanglingLayers(weights, wires = range(num_qubits))
    return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1))

def Variational_classifier(params, x):
    weights = params[0]
    bias1 = params[1]
    bias2 = params[2]
    
    M1, M2 = quantum_circuit(weights, x)
    return [M1 + bias1, M2 + bias2]

In [59]:
def cost_fn(params, X, labels):
    num_labels = len(labels)
    loss = 0
    predictions = [Variational_classifier(params, x) for x in X]
    for l,p in zip(labels, predictions):
        l1, l2, p1, p2 = l[0], l[1], p[0], p[1]
        loss += (p1 - l1)**2 + (p2 - l2)**2
    loss / num_labels
    return loss

In [60]:
def accuracy(labels, predictions):
    acc = 0
    for l,p in zip(labels, hard_predictions):
        if np.abs(l - p) < 1e-2:
            acc = acc + 1
    acc = acc / len(labels)
    return acc

In [61]:
np.random.seed(0)
num_qubits = 4
num_layers = 4
batch_size = 10
lr_adam = 10
total_iterations = 200

In [62]:
def training(num_layers, steps):
    
    weights = 0.01 * np.random.randn(num_layers, num_qubits, 3)
    bias1 = 0.0
    bias2 = 0.0
    params = [weights, bias1, bias2]
    
    opt = qml.NesterovMomentumOptimizer(0.1)
    
    for i in range(total_iterations):
        batch_index = np.random.choice(len(X), size = batch_size)
        
        X_batch = X[batch_index]
        Y_batch = Y[batch_index]
        params, cost_new = opt.step_and_cost(lambda p: cost_fn(params, X_batch, Y_batch), params)  
        
        if i % 10 == 0:
            print("iteration: ", i, " cost: ", cost_new)
        
    return params

In [63]:
X, Y = load_and_process_data()
params = training(num_layers, total_iterations)

iteration:  0  cost:  6.173404219479707
iteration:  10  cost:  5.357248527573184
iteration:  20  cost:  4.559488981620513
iteration:  30  cost:  4.96044624939569
iteration:  40  cost:  6.170163416933162
iteration:  50  cost:  3.1097986795770796
iteration:  60  cost:  5.907569542966045
iteration:  70  cost:  8.346247848119914
iteration:  80  cost:  7.291827405050984
iteration:  90  cost:  5.761092996256702
iteration:  100  cost:  2.7674767903431476
iteration:  110  cost:  6.5596714019530005
iteration:  120  cost:  5.863357625530661
iteration:  130  cost:  7.079423826741692
iteration:  140  cost:  6.222805062000653
iteration:  150  cost:  6.150025538451837
iteration:  160  cost:  7.671461581767906
iteration:  170  cost:  4.042409970477629
iteration:  180  cost:  3.043397690965807
iteration:  190  cost:  4.080831707873828
